### This notebook implements Naive Bayes classification in a privacy-preserving manner. ###
In machine learning, naive Bayes classifiers are a family of simple "probabilistic classifiers" based on applying Bayes' theorem with strong (naive) independence assumptions between the features. More mathematical explaination is provided by https://en.wikipedia.org/wiki/Naive_Bayes_classifier 

Applying Naive Bayes in a veritcally partitioned data scenario, we need to consider two different situations:
1. Both data sites know the target class
2. Only one (part) of data sites know the target class and they are not willing to share
    * Features are categorical values
    * Features are numerical values
    
We provide solutions for both situations in this notebook. 

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from scipy.special import comb, logsumexp
from sklearn.preprocessing import LabelBinarizer

### Situation 1: Naive Bayes - all parties know target class ###
We use Gaussian Naive Bayes to solve numerical features problem. Categorical features can be solved in the same way by using different kernels.
Please check https://scikit-learn.org/stable/modules/classes.html#module-sklearn.naive_bayes to find more possible kernels.
In this notebook, we assume there are two data parties, semi-honest which means they will follow the protocal and still curious about the other parties data. 

############ Due to the var_smoothing variable in Scikit Learn model, np.(X, axis=0).max() needs set the same smoothing variable. I spent the whole day to find why the combined model does not equal to the centralized model ############

In [146]:
var_smoothing_ = 1e-20
var_smoothing_trans = (np.var(df_ctr, axis=0).max()) * var_smoothing_ / (np.var(df_A, axis=0).max())

In [67]:
##### At Data site A #####
df_A_withTar = pd.DataFrame.from_csv('preprocessed_dataFile_A.csv')
TargetClass = df_A_withTar['number_outpatient'] # Target class which is known by both data parties

# Data will be input to the model #
df_A = df_A_withTar.drop(['number_outpatient'], axis=1)

# Fit the Naive Bayes model #
gnb_A1 = GaussianNB(var_smoothing=var_smoothing_trans) #### OMG,WTF,AAAAAAAA, I become crazy to find this!!!!!
gnb_A1.fit(df_A, TargetClass)

# Get the probability estimates #
lh_A1 = gnb_A1._joint_log_likelihood(df_A)
lhl_A1 = logsumexp(lh_A1, axis=1)
PE_A1 = np.exp(lh_A1 - np.atleast_2d(lhl_A1).T)

In [68]:
##### At Data site B #####
df_B = pd.DataFrame.from_csv('preprocessed_dataFile_B.csv')

gnb_B1 = GaussianNB(var_smoothing = var_smoothing_)
gnb_B1.fit(df_B, TargetClass)

lh_B1 = gnb_B1._joint_log_likelihood(df_B)
lhl_B1 = logsumexp(lh_B1, axis=1)
PE_B1 = np.exp(lh_B1 - np.atleast_2d(lhl_B1).T)

In [69]:
##### This part calculation can be done at Trusted Third Party ####
lh_D = lh_A1 + lh_B1 - np.log(gnb_A1.class_prior_)
lhl_D = logsumexp(lh_D, axis=1)

# Outcome is probability estimates #
# The instances will be classified to the class which has the highest probability #  
PE_D1 = np.exp(lh_D - np.atleast_2d(lhl_D).T)

In [70]:
##### Checking the results with centralized dataset #####
df_ctr = pd.concat([df_A, df_B], axis=1)

gnb_C1 = GaussianNB(var_smoothing = var_smoothing_)
gnb_C1.fit(df_ctr, TargetClass)

lh_C1 = gnb_C1._joint_log_likelihood(df_ctr)
lhl_C1 = logsumexp(lh_C1, axis=1)
PE_C1 = np.exp(lh_C1 - np.atleast_2d(lhl_C1).T)
PE_C1

array([[  6.91341644e-10,   4.34641283e-08,   0.00000000e+00, ...,
          0.00000000e+00,   9.35342940e-01,   0.00000000e+00],
       [  2.85991005e-17,   1.56440293e-15,   9.99999395e-01, ...,
          2.98338253e-37,   4.75642956e-07,   0.00000000e+00],
       [  6.65256317e-06,   1.95695013e-04,   0.00000000e+00, ...,
          0.00000000e+00,   3.65514993e-04,   0.00000000e+00],
       ..., 
       [  9.99999972e-01,   2.76898111e-08,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.12750655e-03,   6.50278181e-02,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  8.04122143e-03,   5.63358836e-02,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [71]:
(PE_D1-PE_C1).max()

1.1990408665951691e-14

### Situation 2: Naive Bayes (partially have target class) ###
The challenging part here is to calculate mean and variance values for the party who does not have the target class

##### At Data Site A which has the target class #####
Model can be built locally as target class is available at this data site

In [167]:
TargetClass = df_A_withTar['number_outpatient']
df_A = df_A_withTar.drop(['number_outpatient'], axis=1)

gnb_A2 = GaussianNB(var_smoothing = var_smoothing_trans)
gnb_A2.fit(df_A, TargetClass)

lh_A2 = gnb_A2._joint_log_likelihood(df_A)
lhl_A2 = logsumexp(lh_A2, axis=1)
PE_A2 = np.exp(lh_A2 - np.atleast_2d(lhl_A2).T)

In [168]:
unique_y = np.unique(TargetClass)

class_prior = []
if len(class_prior) == 0:
    # Empirical prior, with sample_weight taken into account
    cnt = Counter(TargetClass)
    for y_i in unique_y:
        class_prior.append(cnt[y_i]/len(TargetClass))

In [169]:
### convert target class to multiple binary classes ###
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(unique_y)
conv_TargetClass = lb.transform(TargetClass)

#### At Data Site B (no TargetClass) ####
Firstly, we need to calculate mean value of each features at data site B based on different classes of Target feature.
Here we need to use secure scalar product to get the mean(s).

##### Starts at data site B #####

In [170]:
X_a = df_B # .transpose()
len_A = len(X_a.columns)

# Generate random numbers and add to data at Data Site A
A_randoms = []
for i in range(0, len_A):
    A_randoms.append(np.random.randint(0,5, len(X_a.iloc[:,i])))
    
C_matrix = [] # C_noises is shared between A and B 
for i in range(0, len_A):
    C_matrix.append(np.random.randint(0,5, (len(X_a.iloc[:,i]), len(X_a.iloc[:,i]))))

Sum_noises_A = [] # which will be sent to B
for i in range(0, len_A):
    Sum_noises_A.append(np.add(X_a.iloc[:,i], np.dot(C_matrix[i], A_randoms[i])))

Data Site A: send __C_matrix__ and __Sum_noises_A__, __A_randoms_Sumset__ to Data Site B

##### Data Site A receive noises from Data site B then do next calculation #####

In [171]:
X_b = pd.DataFrame.from_records(conv_TargetClass)
len_B = len(X_b.columns)
B_divide_set = 10

Sum_coef_B = []
for i in range(0, len_B):
    Sum_noises_temp = []
    for j in range(0, len_A):
        Sum_noises_temp.append(np.dot(C_matrix[j].transpose(), X_b.iloc[:,i])) 
    Sum_coef_B.append(Sum_noises_temp)

B_random_set = []
for i in range(0, len_A):
#     np.random.seed(3)
    B_random_set.append(np.random.randint(0,5, int(len(X_b.iloc[:,0])/B_divide_set))) 

Sum_noises_B = [] # which will be send to A
for n in range(0, len_B):
    B_noise = []
    for i in range(0, len_A):
        B_random_inter = []
        for j in range(0, len(B_random_set[i])): 
            for k in range(0, B_divide_set):
                B_random_inter.append(B_random_set[i][j])
        B_noise.append(Sum_coef_B[n][i] + B_random_inter)
    Sum_noises_B.append(B_noise)

# Add noises dataset A to the dataset B
Sum_noises_AB = []
for i in range(0, len_B):
    Sum_noises_temp = []
    for j in range(0, len_A):
        Sum_noises_temp.append(np.dot(Sum_noises_A[j], X_b.iloc[:,i])) # X_b[:,i]
    Sum_noises_AB.append(Sum_noises_temp)

B sends __Sum_noises_B__ to A

##### Back to Data Site B #####

In [172]:
A_randoms_Sumset = []
for i in range(0, len_A):
    sum_temp = []
    for j in range(0, int(len(X_a)/B_divide_set)):
        temp = 0
        for k in range(0, B_divide_set):
            temp = temp + A_randoms[i][B_divide_set*j + k]
        sum_temp.append(temp)
        
    A_randoms_Sumset.append(sum_temp)

    
Sum_noises_B_Arand = []
for n in range(0, len_B):
    temp = []
    for i in range(0, len_A):
        temp.append(np.dot(A_randoms[i],Sum_noises_B[n][i]))
    Sum_noises_B_Arand.append(temp)

##### Move to Data Site A again --> As A has the target feature, A calculate the final results #####

In [173]:
rand_sums = []
for i in range(0, len_A):
    r_sum = 0
    for j in range(0, len(B_random_set[0])):
        r_sum = r_sum + A_randoms_Sumset[i][j] * B_random_set[i][j]
    rand_sums.append(r_sum)

outcomes = []
for n in range(0, len_B):
    out = []
    for i in range(0, len_A):
        out.append(Sum_noises_AB[n][i] - Sum_noises_B_Arand[n][i] + rand_sums[i]) 
    outcomes.append(out)
    
sum_out = np.array(outcomes)
mean = []
for s in range(0, len(sum_out)):
    mean.append(list(sum_out[s, :]/Counter(TargetClass)[unique_y[s]]))
#     print(list(sum_out[s, :]/Counter(TargetClass)[unique_y[s]]))

In [184]:
# ### check with normal model### mean = theta_
gnb_B2 = GaussianNB(var_smoothing=var_smoothing_)
gnb_B2.fit(df_B, TargetClass)
(gnb_B2.theta_ - mean).max()

0.0

#### Securely compute variance ####
* Now, we have mean(s). Data Site A needs to send mean(s) to Data Site B to calculate variance.
* Let's calculate variance.
* Formula: variance = mean(abs(x-x(mean))**2)

##### At Data Site B: we calculate abs(x-x(mean)) ** 2 #####

In [175]:
list_meanSub2 = []
for i in range(0, len(mean)):
    list_meanSub2.append(np.power(df_B.sub(mean[i], axis='columns'), 2))
B_divide_set = 10

In [176]:
def step_1(X_a, B_divide_set):
    len_A = len(X_a.columns)

    # Generate random numbers and add to data at Data Site A
    A_randoms = []
    for i in range(0, len_A):
        A_randoms.append(np.random.randint(0,5, len(X_a.iloc[:,i])))

    C_matrix = [] # C_noises is shared between A and B 
    for i in range(0, len_A):
        C_matrix.append(np.random.randint(0,5, (len(X_a.iloc[:,i]), len(X_a.iloc[:,i]))))

    Sum_noises_A = [] # which will be sent to B
    for i in range(0, len_A):
        Sum_noises_A.append(np.add(X_a.iloc[:,i], np.dot(C_matrix[i], A_randoms[i])))
        
    A_randoms_Sumset = []
    for i in range(0, len_A):
        sum_temp = []
        for j in range(0, int(len(X_a)/B_divide_set)):
            temp = 0
            for k in range(0, B_divide_set):
                temp = temp + A_randoms[i][B_divide_set*j + k]
            sum_temp.append(temp)

        A_randoms_Sumset.append(sum_temp)
    
    return C_matrix, Sum_noises_A, A_randoms_Sumset, A_randoms, len_A

In [177]:
C_matrix_list = []
Sum_noises_A_list = []
A_randoms_Sumset_list = []
A_randoms_list = []

# Each class has one matrix of abs(x-x(mean))** 2 
# It means Data Site B needs to send n matrix to Data Site A(n = classes)
for i in range(0, len(list_meanSub2)):
    C_matrix, Sum_noises_A, A_randoms_Sumset, A_randoms, len_A = step_1(list_meanSub2[i], B_divide_set)
    C_matrix_list.append(C_matrix)
    Sum_noises_A_list.append(Sum_noises_A)
    A_randoms_Sumset_list.append(A_randoms_Sumset)
    A_randoms_list.append(A_randoms)

##### B Sends N Noised Matrix to Data Site A #####

In [178]:
# Here we assume only one feature is regarded as the target class #
def B_Cal (X_b, len_A, C_matrix, Sum_noises_A):
    Sum_coef_B = []
    for j in range(0, len_A): 
        Sum_coef_B.append(np.dot(C_matrix[j].transpose(), X_b)) # X_b[:,i]

    B_random_set = []
    for i in range(0, len_A):
    #     np.random.seed(3)
        B_random_set.append(np.random.randint(0,5, int(len(X_b)/B_divide_set))) # X_b[:,i]

    Sum_noises_B = [] # which will be send to A
    for i in range(0, len_A):
        B_random_inter = []
        for j in range(0, len(B_random_set[i])): 
            for k in range(0, B_divide_set):
                B_random_inter.append(B_random_set[i][j])
        Sum_noises_B.append(Sum_coef_B[i] + B_random_inter) # B_noise Sum_coef_B[n][i]

    # Add noises dataset A to the dataset B
    Sum_noises_AB = []
    for j in range(0, len_A):
        Sum_noises_AB.append(np.dot(Sum_noises_A[j], X_b)) # X_b[:,i] Sum_noises_temp
    
    return Sum_noises_B, Sum_noises_AB, B_random_set

In [179]:
Sum_noises_B_list = []
Sum_noises_AB_list = []
B_random_set_list = []
for i in range(0, len(conv_TargetClass[0])):
    Sum_noises_B, Sum_noises_AB, B_random_set = B_Cal(conv_TargetClass[:,i], len_A, \
                                                      C_matrix_list[i], Sum_noises_A_list[i])
    Sum_noises_B_list.append(Sum_noises_B)
    Sum_noises_AB_list.append(Sum_noises_AB)
    B_random_set_list.append(B_random_set)

##### Back to Data Site B #####

In [180]:
Sum_noises_B_Arand_list = []
for k in range(0, len(Sum_noises_B_list)):

    Sum_noises_B_Arand = []
    for i in range(0, len_A):
        Sum_noises_B_Arand.append(np.dot(A_randoms_list[k][i],Sum_noises_B_list[k][i])) #temp
        
    Sum_noises_B_Arand_list.append(Sum_noises_B_Arand)

##### Last stage: Go to Data Site A again to calculate the final results (variance) #####

In [181]:
rand_sums_list = []
for n in range(0, len(B_random_set_list)):
    rand_sums = []
    for i in range(0, len_A):
        r_sum = 0
        for j in range(0, len(B_random_set_list[n][0])):
            r_sum = r_sum + A_randoms_Sumset_list[n][i][j] * B_random_set_list[n][i][j]
        rand_sums.append(r_sum)
    rand_sums_list.append(rand_sums)

In [182]:
outcomes_var = []
for n in range(0, len(Sum_noises_AB_list)):
    outcomes = []
    for i in range(0, len_A):
        outcomes.append(Sum_noises_AB_list[n][i] - Sum_noises_B_Arand_list[n][i] + rand_sums_list[n][i]) # out
    outcomes_var.append(outcomes)

In [183]:
sum_out = np.array(outcomes_var)
variance_init = []
for s in range(0, len(sum_out)):
    variance_init.append(list(sum_out[s, :]/Counter(TargetClass)[unique_y[s]]))
    
epsilon = var_smoothing_ * np.var(df_B, axis=0).max() # needs to provide the biggest variance of the whole dataset
variance = np.array(variance_init) + epsilon 

In [185]:
### check with centralized model### variance = sigma_
# gnb_B2.sigma_
# ((variance - gnb_B2.sigma_)).max()

9.4587448984384537e-11

In [150]:
##### Calculation of Probability Estimate #####

In [186]:
def likelihood(df_B, unique_y, class_prior, mean, variance):
    joint_log_likelihood = []
    for i in range(np.size(unique_y)):###
        jointi = np.log(class_prior[i])
        n_ij = - 0.5 * np.sum(np.log(2. * np.pi * variance[i, :]))
        n_ij -= 0.5 * np.sum(((df_B - mean[i, :])**2)/(variance[i, :]), 1)
        joint_log_likelihood.append(jointi + n_ij)

    joint_log_likelihood = np.array(joint_log_likelihood).T
    return joint_log_likelihood

In [187]:
lh_B2 = likelihood(df_B, unique_y, class_prior, np.array(mean), np.array(variance))
lhl_B2 = logsumexp(lh_B2, axis=1)
PE_B2 = np.exp(lh_B2 - np.atleast_2d(lhl_B2).T)

In [188]:
# ### . Check the outcome
# (gnb_B2.predict_proba(df_B)-PE_B2).max()

8.1436760113184903e-10

In [189]:
##### The final step: calculation of probability estimate is better to be done at Trusted Tirth Party ####
lh_D2 = lh_A2 + lh_B2 - np.log(gnb_B2.class_prior_)
lhl_D2 = logsumexp(lh_D2, axis=1)

# Outcome is probability estimates #
# The instances will be classified to the class which has the highest probability #  
PE_D2 = np.exp(lh_D2 - np.atleast_2d(lhl_D2).T)

array([[  2.34600287e-48,   1.47187714e-35,   0.00000000e+00, ...,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  2.86870118e-61,   1.56588251e-48,   1.00000000e+00, ...,
          9.42137512e-43,   1.50357780e-12,   0.00000000e+00],
       [  5.77977279e-41,   1.69659483e-28,   0.00000000e+00, ...,
          0.00000000e+00,   9.99999973e-01,   0.00000000e+00],
       ..., 
       [  3.66234175e-04,   9.99633766e-01,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.36642869e-25,   6.98510839e-13,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  8.62149936e-25,   6.02717224e-13,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [191]:
##### Checking the results with centralized dataset #####
# df_ctr = pd.concat([df_A, df_B], axis=1)

# gnb_C1 = GaussianNB(var_smoothing=var_smoothing_)
# gnb_C1.fit(df_ctr, TargetClass)

# lh_C1 = gnb_C1._joint_log_likelihood(df_ctr)
# lhl_C1 = logsumexp(lh_C1, axis=1)
# PE_C1 = np.exp(lh_C1 - np.atleast_2d(lhl_C1).T)
# (PE_C1-PE_D2).max()

4.2377878983757e-11

### Bayesian Networks ###

In [ ]:
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator
# data = pd.DataFrame(data={'A': [0, 0, 1], 'B': [0, 1, 0], 'C': [1, 1, 0]})
# model = BayesianModel([('A', 'C'), ('B', 'C')])
# cpd_A = MaximumLikelihoodEstimator(model, data).estimate_cpd('A')
# print(cpd_A)

In [ ]:
df_A.head()

In [ ]:
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel
from scipy.stats import chisquare
bdeu_ESS = 5
data = df_A
bdeu = BdeuScore(data, equivalent_sample_size=bdeu_ESS)
k2 = K2Score(data)
bic = BicScore(data)
 
model1 = BayesianModel([('race', 'num_procedures'), ('age', 'num_procedures')])

print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

##### Data Site A with Target Class #####

In [ ]:
def give_state_names(df, feature):
    state_names = dict()
    if isinstance(feature, list): 
        for c in feature:
            values = list(Counter(df[c]).keys())
            values.sort()
            state_names[c] = values
    else:
        values = list(Counter(df[feature]).keys())
        values.sort()
        state_names[feature] = values
        
    return state_names

In [ ]:
state_names = give_state_names(df_A, B_feature)
state_names

In [ ]:
A_feature = 'num_procedures'
state_names_A = give_state_names(df_A, A_feature)

In [ ]:
B_feature = ['race', 'age']
state_names_B = give_state_names(df_A, B_feature)
state_names_B

In [ ]:
# convert target class to binary
def toBinary(unique_y, TargetClass):
    lb = LabelBinarizer()
    lb.fit(unique_y)
    conv_TargetClass = lb.transform(TargetClass)
    return conv_TargetClass

In [ ]:
conv_features_B = []
for b in range(0, len(B_feature)):
    conv_features_B.append(toBinary(state_names_B[B_feature[b]], df_A[B_feature[b]]))

In [ ]:
# Assume B has two features
status_local_matrix = []
if len(conv_features_B) > 1:
    for row in range(0, len(conv_features_B[0])):
        status_local_matrix.append(list(np.concatenate(np.dot((conv_features_B[0][row][np.newaxis]).T, \
                                                  conv_features_B[1][row][np.newaxis]))))

In [ ]:
##### Secure scalar product ##### status_local_matrix & TargetClass_A
### status_local_matrix has to be sent to Data Site A ###
X_a = pd.DataFrame.from_records(status_local_matrix) # .transpose()
len_A = len(X_a.columns)

# Generate random numbers and add to data at Data Site A
A_randoms = []
for i in range(0, len_A):
    A_randoms.append(np.random.randint(0,5, len(X_a.iloc[:,i])))
    
C_matrix = [] # C_noises is shared between A and B 
for i in range(0, len_A):
    C_matrix.append(np.random.randint(0,5, (len(X_a.iloc[:,i]), len(X_a.iloc[:,i]))))

Sum_noises_A = [] # which will be sent to B
for i in range(0, len_A):
    Sum_noises_A.append(np.add(X_a.iloc[:,i], np.dot(C_matrix[i], A_randoms[i])))

In [ ]:
##### At Data Site A #####
X_b = pd.DataFrame.from_records(toBinary(state_names_A[A_feature], df_A[A_feature]))
len_B = len(X_b.columns)
B_divide_set = 10

Sum_coef_B = []
for i in range(0, len_B):
    Sum_noises_temp = []
    for j in range(0, len_A):
        Sum_noises_temp.append(np.dot(C_matrix[j].transpose(), X_b.iloc[:,i])) 
    Sum_coef_B.append(Sum_noises_temp)

B_random_set = []
for i in range(0, len_A):
#     np.random.seed(3)
    B_random_set.append(np.random.randint(0,5, int(len(X_b.iloc[:,0])/B_divide_set))) 

Sum_noises_B = [] # which will be send to A
for n in range(0, len_B):
    B_noise = []
    for i in range(0, len_A):
        B_random_inter = []
        for j in range(0, len(B_random_set[i])): 
            for k in range(0, B_divide_set):
                B_random_inter.append(B_random_set[i][j])
        B_noise.append(Sum_coef_B[n][i] + B_random_inter)
    Sum_noises_B.append(B_noise)

# Add noises dataset A to the dataset B
Sum_noises_AB = []
for i in range(0, len_B):
    Sum_noises_temp = []
    for j in range(0, len_A):
        Sum_noises_temp.append(np.dot(Sum_noises_A[j], X_b.iloc[:,i])) # X_b[:,i]
    Sum_noises_AB.append(Sum_noises_temp)

In [ ]:
##### Back to Data Site B #####
A_randoms_Sumset = []
for i in range(0, len_A):
    sum_temp = []
    for j in range(0, int(len(X_a)/B_divide_set)):
        temp = 0
        for k in range(0, B_divide_set):
            temp = temp + A_randoms[i][B_divide_set*j + k]
        sum_temp.append(temp)
        
    A_randoms_Sumset.append(sum_temp)

    
Sum_noises_B_Arand = []
for n in range(0, len_B):
    temp = []
    for i in range(0, len_A):
        temp.append(np.dot(A_randoms[i],Sum_noises_B[n][i]))
    Sum_noises_B_Arand.append(temp)

In [ ]:
##### At Data Site A to calculate final result #####
rand_sums = []
for i in range(0, len_A):
    r_sum = 0
    for j in range(0, len(B_random_set[0])):
        r_sum = r_sum + A_randoms_Sumset[i][j] * B_random_set[i][j]
    rand_sums.append(r_sum)

outcomes = []
for n in range(0, len_B):
    out = []
    for i in range(0, len_A):
        out.append(Sum_noises_AB[n][i] - Sum_noises_B_Arand[n][i] + rand_sums[i]) 
    outcomes.append(out)

In [ ]:
# outcomes_df = pd.DataFrame.from_records(outcomes)
header_list = []
for i in range(0, len(B_feature)):
    header_list.append(state_names_B[B_feature[i]])
header = pd.MultiIndex.from_product(header_list,names=B_feature)
outcomes_state_counts_df = pd.DataFrame.from_records(outcomes, columns=header)
outcomes_state_counts_df.index.name = A_feature

In [ ]:
# K2: 
# k2 = K2Score(data)
# print(k2.local_score('num_procedures', parents=['race', 'age']))

from math import lgamma, log
    
var_states = state_names_A[A_feature]
var_cardinality = len(var_states)
state_counts = outcomes_state_counts_df

score = 0
for parents_state in outcomes_state_counts_df:  # iterate over df columns (only 1 if no parents)
    
    conditional_sample_size = sum(outcomes_state_counts_df[parents_state])
    score += lgamma(var_cardinality_A) - lgamma(conditional_sample_size + var_cardinality_A)

    for state in var_states:
        if outcomes_state_counts_df[parents_state][state] > 0:
            score += lgamma(outcomes_state_counts_df[parents_state][state] + 1)
print(score)

# Done #

In [ ]:
def randomize(feature, name):
    if type(feature) == pd.core.series.Series:
        rep_var = []
        uni_var = list(Counter(feature).keys())
        random_var = np.random.sample(len(uni_var)) #list(Counter(data['WEALTH_INDEX']).keys()) #
        print('%s unique values are in the feature %s' %(len(Counter(feature)), name))
        for i in feature:
            position = uni_var.index(i)
            rep_var.append(random_var[position])
    else:
        rep_var = []
        colomn = list(Counter(feature))
        for col in colomn:
            slg_var = []
            uni_var = list(Counter(feature[col]).keys())
            random_var = np.random.sample(len(uni_var)) #list(Counter(data['WEALTH_INDEX']).keys()) #
            print('%s unique values are in the features -- %s' %(len(uni_var), col))
            for i in feature[col]:
                position = uni_var.index(i)
                slg_var.append(random_var[position])
            rep_var.append(slg_var)
    return rep_var

def transform(rep_var):
    var_states = list(Counter(rep_var).keys()) # A site
    var_list = []
    for v in var_states:
        var_num = []
        for i in rep_var:
            if i == v:
                var_num.append(1)
            else:
                var_num.append(0)
        var_list.append(var_num) # send to B site
    var_cardinality = len(var_list) # send to B site
    return var_states, var_list, var_cardinality

In [ ]:
rep_varA = randomize(df_A[A_feature], A_feature)
rep_varB = []
for b in B_feature:
    rep_varB.append(randomize(df_A[b], b))
rep_varB = np.array(rep_varB)
var_states, var_list_A, var_cardinality_A = transform(rep_varA)

In [ ]:
A_feature = 'num_procedures'
B_feature = ['race', 'age']
rep_varA = randomize(df_A[A_feature], A_feature)
rep_varB = []
for b in B_feature:
    rep_varB.append(randomize(df_A[b], b))
rep_varB = np.array(rep_varB)
var_states, var_list_A, var_cardinality_A = transform(rep_varA)

In [ ]:
def get_state_counts(var_list_A, rep_varB, B_feature, b):
    state_counts = []
    
    for var in var_list_A: 
        cnt_varB = []
        list_varB = []
        for i in range(0, len(var)):
            if var[i] == 1:
                cnt_varB.append(list(rep_varB[:, i]))
        # Second layer
        list_sec = []
        key_sec = []
        cnt_varB = pd.DataFrame.from_records(cnt_varB)
        cnt_varB.columns = B_feature
        sec_keys = Counter(cnt_varB[B_feature[b]]).keys() ######
        for key in sec_keys:
            dic_counter = Counter(cnt_varB.loc[cnt_varB[B_feature[b]] == key][B_feature[1]])
            for k in dic_counter.keys():
                if k not in key_sec: 
                    key_sec.append(k)
            list_sec.append(dic_counter)
        # Counter 的顺序要一样
        list_key = list(Counter(cnt_varB[B_feature[b]]).keys())
        list_values = list(Counter(cnt_varB[B_feature[b]]).values())
        for r in Counter(rep_varB[b]).keys():
            if r in list_key: 
                list_varB.append(list_values[list_key.index(r)])
            else:
                list_varB.append(0)
        state_counts.append(list_varB)
        
    return state_counts

In [ ]:
state_counts_list = []
for b in range(0, len(B_feature)):
    state_counts = get_state_counts(var_list_A, rep_varB, B_feature, b)
    state_counts_list.append(state_counts)

In [ ]:
var_cardinality_A

In [ ]:
state_counts_df_list = []
for s in range(0, len(state_counts_list)):
    state_counts_df = pd.DataFrame.from_records(state_counts_list[s])
    state_counts_df.columns = list(Counter(rep_varB[s]).keys())
    state_counts_df.index = var_states
    state_counts_df_list.append(state_counts_df)

In [ ]:
# K2: 
k2 = K2Score(data)
# model1 = BayesianModel([('num_procedures', 'race')])
# print(k2.score(model1))
print(k2.local_score('num_procedures', parents=['age', 'race']))

from math import lgamma, log
score = 0
for parents_state in state_counts_df_list[1]:  # iterate over df columns (only 1 if no parents)
    conditional_sample_size = sum(state_counts_df_list[1][parents_state])

    score += lgamma(var_cardinality_A) - lgamma(conditional_sample_size + var_cardinality_A)

    for state in var_states:
        if state_counts_df_list[1][parents_state][state] > 0:
            score += lgamma(state_counts_df_list[1][parents_state][state] + 1)
print(score)

In [ ]:
state_names = give_state_names(df_A)
variable = 'num_procedures'
parents = ['age', 'race']

parents_states = [state_names[parent] for parent in parents]

# count how often each state of 'variable' occured, conditional on parents' states
state_count_data = df_A.groupby([variable] + parents).size().unstack(parents)

In [ ]:
if not isinstance(state_count_data.columns, pd.MultiIndex):
    state_count_data.columns = pd.MultiIndex.from_arrays([state_count_data.columns])

In [ ]:
row_index = state_names[variable]
column_index = pd.MultiIndex.from_product(parents_states, names=parents)
state_counts = state_count_data.reindex(index=row_index, columns=column_index).fillna(0)

In [ ]:
k2.state_counts('num_procedures', parents = ['age', 'race'])

#### Secure Sum ####

In [ ]:
D = np.array([[0,0,0,0,0,0],[2,2,2,2,2,2],[3,3,3,3,3,3]])

k = 2
n = 3

In [ ]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [ ]:
DB = []
for i in range(0, len(D)):
    DB.append(chunkIt(D[i], k))
DB

In [ ]:
s = np.zeros((3, 2, 3))

for rc in range(k, 0, -1):
    for j in range(0, k):
        for i in range(0, n):
            s[i][j] = DB[i][j] + s[i][j]
s